# Final project for the Coursera course IBM Applied Data Science Capstone
## *Work in progress*
### Tamara Thornquist

In [35]:
# import all necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

import nbconvert

Folium installed
Libraries imported.


In [36]:
CLIENT_ID = 'SS2DCCVHW3ZMK3HYPBSS1TQU3MMTHTVPVZ2CLF0AZHTGECOK' # Foursquare ID
CLIENT_SECRET = 'BMV3QEC3ZHW13S3BNKQVONLDVRXWTLYIP0BIEPJ51PVLTMOV' # Foursquare Secret
ACCESS_TOKEN = 'ILKUZ2UM41UMBTMAV1QFL2UOIPCB1FTCDYA5PK4ZAQDWK3F3' # FourSquare Access Token
VERSION = '20200204'
LIMIT = 30

# 1. Data loading and preparation

#### Find coordinates for Stockholm

In [37]:
address = 'Stockholm, Sweden'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Stockholm's coordinates are",latitude,longitude)

Stockholm's coordinates are 59.3251172 18.0710935


#### Display Stockholm's map

In [38]:
# Stockholm's map
stockholm_map=folium.Map([latitude, longitude], zoom_start=13)
stockholm_map

#### Construct a list of boroughs (districts) in Stockholm. Scrap the web and construct a dataframe with boroughs in Stockholm

In [39]:
import requests

#### The latest list of boroughs in Stockholm is available on the official web page of the Stockholm county. Send the request to the web page and download the HTML document

In [40]:
page=requests.get('https://vaxer.stockholm/omraden/')

#### Use BeautifulSoup library to parse the document

In [41]:
from bs4 import BeautifulSoup

#### Create an instance of the BeautifulSoup class to parse a document

In [42]:
soup=BeautifulSoup(page.content, 'html.parser')

In [43]:
# one can now print out the HTML content of the page, formatted nicely, using the prettify method on the BeautifulSoup object:
#print(soup.prettify())

In [44]:
# As all the tags are nested, one can move through the structure one level at a time. 
# Select all the elements at the top level of the page using the children property of soup. 
# Since children returns a list generator, call the list function on it:
#list(soup.children)[:4]

In [45]:
# type of the elements
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag]

In [46]:
# all of the items are BeautifulSoup objects. Doctype object contains information about the type of the document. 
# NavigableString represents text found in the HTML document. A Tag object contains other nested tags. 
# one can select the html tag and its children by taking the third item in the list:
html=list(soup.children)[3]

In [47]:
type(html)

bs4.element.Tag

In [48]:
# find chidlren inside html tag
#list(html.children)

In [49]:
# use a tag and class to select info
district=html.find('a', class_='block-link__title header-list__title').get_text()
district

'Bromma stadsdelsområde'

#### Select necessary information and save it as a dataframe

In [50]:
# use a tag and class to select info, save as a list
district_list=[d.get_text() for d in html.find_all('a', class_='block-link__title header-list__title')]

In [51]:
district_list

['Bromma stadsdelsområde',
 'Enskede-Årsta-Vantörs stadsdelsområde',
 'Farsta stadsdelsområde',
 'Hägersten-Älvsjö stadsdelsområde',
 'Hässelby-Vällingby stadsdelsområde',
 'Kungsholmens stadsdelsområde',
 'Norrmalms stadsdelsområde',
 'Rinkeby-Kista stadsdelsområde',
 'Skarpnäcks stadsdelsområde',
 'Skärholmens stadsdelsområde',
 'Spånga-Tensta stadsdelsområde',
 'Södermalms stadsdelsområde',
 'Östermalms stadsdelsområde']

In [52]:
# construct dataframe with districts' names
df_district=pd.DataFrame({'District Name':district_list})

In [53]:
df_district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   District Name  13 non-null     object
dtypes: object(1)
memory usage: 232.0+ bytes


#### Dataframe with districts' names

In [54]:
df_district

,District Name
0,Bromma stadsdelsområde
1,Enskede-Årsta-Vantörs stadsdelsområde
2,Farsta stadsdelsområde
3,Hägersten-Älvsjö stadsdelsområde
4,Hässelby-Vällingby stadsdelsområde
5,Kungsholmens stadsdelsområde
6,Norrmalms stadsdelsområde
7,Rinkeby-Kista stadsdelsområde
8,Skarpnäcks stadsdelsområde
9,Skärholmens stadsdelsområde


#### Remove "stadsdelsområde" from the districts' names

In [55]:
# first, remove 'stadsdelsområde'
# by some reason if I pass ' stadsdelsområde' with a space before 'stadsdelsområde', districts that 
# are made up of only one name get only the first letter left.
df_district['District Name']=df_district['District Name'].map(lambda x:x.rstrip('stadsdelsområde'))

In [56]:
# then, remove ' ' at the end of some districts' names, by default removes white space characters at the rhs.
# however, if I do not specify ' ' nothing happens, so better to specify that it is white space that I want to trim. 
df_district['District Name']=df_district['District Name'].map(lambda x:x.rstrip(' '))

In [57]:
# remove 's' at the end of some districts' names
df_district['District Name']=df_district['District Name'].map(lambda x:x.rstrip('s'))

#### Final dataframe with districts' names

In [58]:
df_district

,District Name
0,Bromma
1,Enskede-Årsta-Vantör
2,Farsta
3,Hägersten-Älvsjö
4,Hässelby-Vällingby
5,Kungsholmen
6,Norrmalm
7,Rinkeby-Kista
8,Skarpnäck
9,Skärholmen


#### Add coordinates for the districts

In [59]:
import json

In [60]:
# Coordinate for Stockholm municipality neighbourhoods.geojson file for Stockholm
with open('/Users/tamarathornquist/Dropbox/Coursera Applied Data Science Capstone/neighbourhoods.geojson') as geojson:
    nei_geojson=json.load(geojson)

In [61]:
coordinates_districts=nei_geojson['features']

In [62]:
coordinates_districts[0]

{'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[17.974196, 59.313997],
     [17.97834, 59.320505],
     [17.97886, 59.320876],
     [17.983636, 59.324282],
     [17.984575, 59.324952],
     [17.989728, 59.328625],
     [17.991126, 59.329644],
     [17.994024, 59.332873],
     [17.994283, 59.333812],
     [17.994532, 59.334714],
     [17.994274, 59.3362],
     [17.995171, 59.337582],
     [17.998763, 59.340125],
     [18.001896, 59.34087],
     [18.004471, 59.341482],
     [18.008462, 59.341066],
     [18.011355, 59.340895],
     [18.013732, 59.34069],
     [18.014951, 59.340453],
     [18.015844, 59.340103],
     [18.016116, 59.340048],
     [18.016728, 59.339924],
     [18.018273, 59.339749],
     [18.02047, 59.339783],
     [18.022796, 59.339819],
     [18.024427, 59.339578],
     [18.025253, 59.339192],
     [18.031122, 59.337815],
     [18.037538, 59.336606],
     [18.039965, 59.33636],
     [18.044381, 59.335675],
     [18.045333, 59.33545],
     [18

In [63]:
columns=['District Name', ' Latitude', 'Longitude']

In [64]:
stock_coord=pd.DataFrame(columns=columns)

In [65]:
stock_coord

,District Name,Latitude,Longitude
